## Shade Computation

In this notebook, we will use GPU to compute the shade for each potetial tree site. We use the LiDAR-derived digital Surface Model (DSM) to compute the shade. By creating a patch for each potential tree site, we can compute:

- S_original: The original shade of the patch using the DSM.
- S_planted: The shade of the patch after planting a tree using `apply_tree_to_dsm()`.
- S_unique: The unique shade of the patch after planting a tree, which is the difference between S_planted and S_original.
- S_unique_on_edge: The unique shade of the patch on the edges of the road network graph, which is the vectorized unique shade that intersects with the road network edge gdf.

By moving two patches using their coordinates, we can compute the overlap between two potential tree sites:

- S_overlap: The overlap shade between two patches, which is the intersection of their unique shades.
- S_overlap_on_edge: The overlap shade on the edges of the road network graph, which is the vectorized overlap shade that intersects with the road network edge gdf.


In [ ]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from tqdm import tqdm, trange
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from rasterio.merge import merge
from collections import defaultdict
import pandas as pd
import pickle
import networkx as nx
from rasterio.windows import Window
from shapely.geometry import shape, box
from shapely.ops import unary_union
import rasterio.features
import gc
  

print("%d device(s) found." % cuda.Device.count())
for ordinal in range(cuda.Device.count()):
    dev = cuda.Device(ordinal)
    print ("Device #%d: %s" % (ordinal, dev.name()))
print (cuda)

# -------------------- PARAMETERS --------------------
# TILE_DIR = "/mnt/e/Capstone/data/tiles_output/test"
TILE_DIR = "/mnt/e/Capstone/data/tiles_output/dsm"
SITES_FP = "/mnt/e/Capstone/data/final/sites_tree.geojson"
FINAL_GEOJSON = "data/final/final_sites.geojson"
STACK_OUT_DIR = "data/tiles_output/stacks"

os.makedirs("data/final/overlap_tile", exist_ok=True)
os.makedirs("data/final/sites_tile", exist_ok=True)
os.makedirs("data/final/shade_on_edge_tile", exist_ok=True)
os.makedirs("data/final/overlap_shade_on_edge_tile", exist_ok=True)

PATCH_SIZE = 512
HALF_PATCH = PATCH_SIZE // 2
PIXEL_SIZE = 1.0  # in feet
# 2 pm, July 1st
AZIMUTH = 243.9  # degrees
ELEVATION = 60.38  # degrees
EDGE_HEIGHT_FRACTION = 0.3  # 30% height at canopy edge

sites_gdf = gpd.read_file(SITES_FP)

# Add columns for your shade stats
for col in ["shade_area","overlap_with_existing","unique_shade_area","overlap_with_new_trees"]:
    sites_gdf[col] = 0.0

with open("/mnt/e/Capstone/data/sites/test/G_ideal.pkl", "rb") as f:
    G_ideal = pickle.load(f)

# ----- Nodes DataFrame -----
nodes = [
    {"node": n, **attrs}
    for n, attrs in G_ideal.nodes(data=True)
]
nodes_df = pd.DataFrame(nodes)
origins = nodes_df.loc[nodes_df["visit_count"] != 0].reset_index(drop=True)

edges = [
    {"origin_node": u, "dest_node": v, **data}
    for u, v, data in G_ideal.edges(data=True)
]
edges_df = pd.DataFrame(edges)
edges_gdf = gpd.GeoDataFrame(edges_df, geometry="geometry", crs=sites_gdf.crs)

# CUDA Kernel
CUDA_KERNEL = """
#define PI 3.1415926

__global__ void calculate_shadow(float *dsm, bool *shadow, int width, int height, float cell_size, float sun_azimuth, float sun_elevation) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    // Check if the thread is within the bounds of the image
    if (x >= width || y >= height)
        return;

    int idx = y * width + x;

    // Convert sun azimuth and elevation to radians
    float azimuth_rad = sun_azimuth * 3.14159 / 180.0;
    float elevation_rad = sun_elevation * 3.14159 / 180.0;

    // Direction of the shadow ray based on sun azimuth
    float dx = sin(azimuth_rad);
    float dy = -cos(azimuth_rad);
    float dz = tan(elevation_rad);

    // Initial height at the current pixel
    float initial_height = dsm[idx];
    bool in_shadow = false;

    // Trace the shadow ray
    for (float t = cell_size; t < 2000.0f; t += cell_size) { // Limit tracing distance
        int x_offset = x + int(dx * t / cell_size);
        int y_offset = y + int(dy * t / cell_size);

        if (x_offset < 0 || x_offset >= width || y_offset < 0 || y_offset >= height)
            break;  // Out of bounds

        int offset_idx = y_offset * width + x_offset;

        // Calculate height along the ray
        float height_along_ray = initial_height + dz * t;

        // Check if there is a higher point along the path
        if (dsm[offset_idx] > height_along_ray) {
            in_shadow = true;
            break;
        }
    }

    // Store shadow result: 1 for shadow, 0 for no shadow
    shadow[idx] = in_shadow;
}
"""

mod = SourceModule(CUDA_KERNEL)
shadow_kernel = mod.get_function("calculate_shadow")

# -------------------- FUNCTIONS --------------------
# def apply_tree_to_dsm(dsm_patch, cx, cy, height, canopy_radius):
#     Y, X = np.ogrid[:dsm_patch.shape[0], :dsm_patch.shape[1]]
#     dist = np.sqrt((X - cx)**2 + (Y - cy)**2)
#     mask = dist <= canopy_radius
#     # dome profile
#     profile = height * np.sqrt(1 - (dist[mask] / canopy_radius)**2)
#     patched = dsm_patch.copy()
#     patched[mask] += profile
#     return patched
def apply_tree_to_dsm(dsm_patch, cx, cy, height, canopy_radius):
    Y, X = np.ogrid[:dsm_patch.shape[0], :dsm_patch.shape[1]]
    dist = np.sqrt((X - cx)**2 + (Y - cy)**2)
    mask = dist <= canopy_radius

    # 计算新树冠球的高度profile
    profile = height * np.sqrt(1 - (dist[mask] / canopy_radius)**2)
    patched = dsm_patch.copy()

    # 树冠高度分布加到基底（种树点下方DSM的高度 a）
    base_height = dsm_patch[int(cy), int(cx)]
    tree_heights = base_height + profile  # 新树的实际高程

    # 替换DSM，取最大值
    patched[mask] = np.maximum(dsm_patch[mask], tree_heights)
    return patched


def compute_shadow(dsm_patch, azimuth, elevation):
    h, w = dsm_patch.shape
    flat_dsm = dsm_patch.astype(np.float32).ravel()
    shadow_result = np.zeros_like(flat_dsm, dtype=np.bool_)
    dsm_gpu = cuda.mem_alloc(flat_dsm.nbytes)
    shadow_gpu = cuda.mem_alloc(shadow_result.nbytes)

    cuda.memcpy_htod(dsm_gpu, flat_dsm)
    cuda.memcpy_htod(shadow_gpu, shadow_result)

    block = (16, 16, 1)
    grid = ((w + 15) // 16, (h + 15) // 16)

    shadow_kernel(dsm_gpu, shadow_gpu,
        np.int32(w), np.int32(h),
        np.float32(PIXEL_SIZE),
        np.float32(azimuth), np.float32(elevation),
        block=block, grid=grid)

    cuda.memcpy_dtoh(shadow_result, shadow_gpu)
    return shadow_result.reshape((h, w))

def compute_unique_shade_area(unique_shade_mask, transform, edges_gdf):
    """
    Vectorize unique shade mask and compute intersection with edges
    Returns list of (edge_id, intersection_length) tuples
    """
    if not unique_shade_mask.any():
        return []
    
    shapes = rasterio.features.shapes(
        unique_shade_mask.astype("uint8"),
        mask=unique_shade_mask,
        transform=transform
    )
    polys = [shape(geom) for geom, val in shapes if val == 1]
    if not polys:
        return []
    
    shadow_union = unary_union(polys)
    
    # Get bounding box for spatial filtering
    minx, miny, maxx, maxy = shadow_union.bounds
    edges_in_patch = edges_gdf.cx[minx:maxx, miny:maxy]
    
    edge_intersections = []
    for eidx, erow in edges_in_patch.iterrows():
        inter = erow.geometry.intersection(shadow_union)
        if not inter.is_empty:
            edge_intersections.append((eidx, inter.length))
    
    return edge_intersections


1 device(s) found.
Device #0: NVIDIA GeForce RTX 3070
<module 'pycuda.driver' from '/home/ubuntu24/miniforge3/envs/spatial/lib/python3.12/site-packages/pycuda/driver.py'>


In [ ]:

# Sort tiles
tile_files = sorted(f for f in os.listdir(TILE_DIR) if f.endswith(".tif"))

for tile_file in tqdm(tile_files, desc="Processing tiles"):
    tile_id = os.path.splitext(tile_file)[0].split("_")[-1]
    tile_fp = os.path.join(TILE_DIR, tile_file)

    # Collectors for this tile
    shade_on_edge_records = []
    overlap_records = []
    overlap_shade_on_edge_records = []

    with rasterio.open(tile_fp) as dsm_ds:
        dsm = dsm_ds.read(1, out_dtype="float32")
        transform = dsm_ds.transform
        height, width = dsm.shape

        # STEP 1: Calculate baseline shadow mask for the entire tile (once)
        print(f"Computing baseline shadow for tile {tile_id}")
        baseline_shadow_tile = compute_shadow(dsm, AZIMUTH, ELEVATION)

        # Define tile bounds for site selection
        full_bounds = rasterio.windows.bounds(
            Window(500, 500, 10000, 10000),
            transform
        )
        tile_geom = box(*full_bounds)

        # Select sites in this tile
        tile_sites = sites_gdf[sites_gdf.intersects(tile_geom)].copy()
        if tile_sites.empty:
            continue

        # Storage for patch data
        site_data = {}  # {site_id: {'unique_mask': array, 'px': int, 'py': int, 'patch_transform': transform}}

        # STEP 2-4: Process each tree site
        for i in trange(len(tile_sites), desc=f"Tile {tile_id} - Processing Trees", leave=False):
            site = tile_sites.iloc[i]
            site_id = site.name
            x, y = site.geometry.x, site.geometry.y
            px, py = map(int, ~transform * (x, y))

            # Check if patch is within bounds
            if px < HALF_PATCH or py < HALF_PATCH or px > width - HALF_PATCH or py > height - HALF_PATCH:
                continue

            # STEP 2: Create 512x512 patch and extract baseline shade
            dsm_patch = dsm[py - HALF_PATCH: py + HALF_PATCH,
                            px - HALF_PATCH: px + HALF_PATCH]
            
            baseline_shadow_patch = baseline_shadow_tile[
                py - HALF_PATCH: py + HALF_PATCH,
                px - HALF_PATCH: px + HALF_PATCH
            ]

            # STEP 3: Simulate new tree and calculate unique shade
            tree_dsm = apply_tree_to_dsm(
                dsm_patch.copy(),
                HALF_PATCH, HALF_PATCH,
                site["height"], site["canopy_radius"]
            )
            
            tree_shadow = compute_shadow(tree_dsm, AZIMUTH, ELEVATION)
            
            # Unique shade = new shade - baseline shade
            unique_shade_mask = tree_shadow & ~baseline_shadow_patch

            # Store patch data for overlap calculations
            window = Window(px - HALF_PATCH, py - HALF_PATCH, PATCH_SIZE, PATCH_SIZE)
            patch_transform = rasterio.windows.transform(window, transform)
            
            site_data[site_id] = {
                'unique_mask': unique_shade_mask,
                'px': px,
                'py': py,
                'patch_transform': patch_transform
            }

            # STEP 4: Calculate shade on edges for this tree
            edge_intersections = compute_unique_shade_area(unique_shade_mask, patch_transform, edges_gdf)
            for edge_id, length in edge_intersections:
                shade_on_edge_records.append((site_id, edge_id, length))

            # Update site statistics
            unique_area = unique_shade_mask.sum() * PIXEL_SIZE**2
            overlap_old = (tree_shadow & baseline_shadow_patch).sum() * PIXEL_SIZE**2
            sites_gdf.at[site_id, "unique_shade_area"] = unique_area
            sites_gdf.at[site_id, "overlap_with_existing"] = overlap_old
            sites_gdf.at[site_id, "shade_area"] = unique_area + overlap_old

        # STEP 5-7: Calculate pairwise overlaps
        site_ids = list(site_data.keys())
        for i in trange(len(site_ids), desc=f"Tile {tile_id} - Pairwise Overlaps", leave=False):
            site_i = site_ids[i]
            data_i = site_data[site_i]
            
            for j in range(i + 1, len(site_ids)):
                site_j = site_ids[j]
                data_j = site_data[site_j]

                # STEP 5: Check if patches can overlap
                dx = data_j['px'] - data_i['px']
                dy = data_j['py'] - data_i['py']
                
                if abs(dx) >= PATCH_SIZE or abs(dy) >= PATCH_SIZE:
                    continue  # Patches too far apart

                # Calculate overlap region in each patch's coordinate system
                if dx >= 0:
                    xi0, xi1 = dx, PATCH_SIZE
                    xj0, xj1 = 0, PATCH_SIZE - dx
                else:
                    xi0, xi1 = 0, PATCH_SIZE + dx
                    xj0, xj1 = -dx, PATCH_SIZE

                if dy >= 0:
                    yi0, yi1 = dy, PATCH_SIZE
                    yj0, yj1 = 0, PATCH_SIZE - dy
                else:
                    yi0, yi1 = 0, PATCH_SIZE + dy
                    yj0, yj1 = -dy, PATCH_SIZE

                
                if xi1 <= xi0 or yi1 <= yi0:
                    continue  # No valid overlap region

                # STEP 6: Extract overlapping regions and compute intersection
                mask_i = data_i['unique_mask'][yi0:yi1, xi0:xi1]
                mask_j = data_j['unique_mask'][yj0:yj1, xj0:xj1]
                
                # Overlap mask: both trees cast shade at the same location
                overlap_mask = mask_i & mask_j
                overlap_area = overlap_mask.sum() * PIXEL_SIZE**2

                if overlap_area > 0:
                    overlap_records.append((site_i, site_j, overlap_area))

                    # STEP 7: Calculate overlap shade on edges
                    # Create transform for the overlap region (use patch i's coordinate system)
                    overlap_window = Window(
                        data_i['px'] - HALF_PATCH + xi0,
                        data_i['py'] - HALF_PATCH + yi0,
                        xi1 - xi0, yi1 - yi0
                    )
                    overlap_transform = rasterio.windows.transform(overlap_window, transform)
                    
                    # Compute edge intersections for overlap
                    edge_intersections = compute_unique_shade_area(overlap_mask, overlap_transform, edges_gdf)
                    for edge_id, length in edge_intersections:
                        overlap_shade_on_edge_records.append((site_i, site_j, edge_id, length))

        # Save results for this tile
        if overlap_records:
            df_overlap = pd.DataFrame(overlap_records, columns=["i", "j", "overlap_area"])
            df_overlap.to_csv(f"data/final/overlap_tile/overlap_tile_{tile_id}.csv", index=False)

        if shade_on_edge_records:
            df_shade_edge = pd.DataFrame(shade_on_edge_records, columns=["site_id", "edge_id", "shade_length"])
            df_shade_edge.to_csv(f"data/final/shade_on_edge_tile/shade_on_edge_tile_{tile_id}.csv", index=False)

        if overlap_shade_on_edge_records:
            df_overlap_shade_edge = pd.DataFrame(
                overlap_shade_on_edge_records,
                columns=["site_i", "site_j", "edge_id", "overlap_shade_length"]
            )
            df_overlap_shade_edge.to_csv(
                f"data/final/overlap_shade_on_edge_tile/overlap_shade_on_edge_tile_{tile_id}.csv",
                index=False
            )

        # Save tile sites
        tile_sites_out = sites_gdf.loc[tile_sites.index].copy()
        tile_sites_out["original_index"] = tile_sites_out.index
        tile_sites_out.to_file(f"data/final/sites_tile/sites_tile_{tile_id}.geojson", driver="GeoJSON")

        print(f"Saved tile {tile_id} data")

print("Processing complete!")

Then, we can combine all the data by tiles into a single gdf or df, and store locally.

In [4]:
import os
import pandas as pd

# Folder where individual overlap CSVs are saved
overlap_folder = "data/final/overlap_tile"

# Collect all filenames
overlap_files = sorted([os.path.join(overlap_folder, f) 
                        for f in os.listdir(overlap_folder) 
                        if f.endswith(".csv")])

# Read and concatenate
overlap_dfs = []
for f in overlap_files:
    df = pd.read_csv(f)
    overlap_dfs.append(df)

# Combine all overlaps
df_all_overlap = pd.concat(overlap_dfs, ignore_index=True)

# Save the merged overlap file
df_all_overlap.to_csv("data/final/tree_pairwise_overlap.csv", index=False)

print(f"Combined {len(overlap_files)}")


Combined 58


In [5]:
import os
import pandas as pd

# Folder where individual overlap CSVs are saved
overlap_folder = "data/final/shade_on_edge_tile"

# Collect all filenames
overlap_files = sorted([os.path.join(overlap_folder, f) 
                        for f in os.listdir(overlap_folder) 
                        if f.endswith(".csv")])

# Read and concatenate
overlap_dfs = []
for f in overlap_files:
    df = pd.read_csv(f)
    overlap_dfs.append(df)

# Combine all overlaps
df_all_overlap = pd.concat(overlap_dfs, ignore_index=True)

# Save the merged overlap file
df_all_overlap.to_csv("data/final/shade_on_edge.csv", index=False)

print(f"Combined {len(overlap_files)}")


Combined 58


In [6]:
import os
import pandas as pd

# Folder where individual overlap CSVs are saved
overlap_folder = "data/final/overlap_shade_on_edge_tile"

# Collect all filenames
overlap_files = sorted([os.path.join(overlap_folder, f) 
                        for f in os.listdir(overlap_folder) 
                        if f.endswith(".csv")])

# Read and concatenate
overlap_dfs = []
for f in overlap_files:
    df = pd.read_csv(f)
    overlap_dfs.append(df)

# Combine all overlaps
df_all_overlap = pd.concat(overlap_dfs, ignore_index=True)

# Save the merged overlap file
df_all_overlap.to_csv("data/final/overlap_shade_on_edge.csv", index=False)

print(f"Combined {len(overlap_files)}")

Combined 56


In [ ]:
import geopandas as gpd

# Folder where individual site GeoJSONs are saved
sites_folder = "data/final/sites_tile"

# Collect all filenames
sites_files = sorted([os.path.join(sites_folder, f) 
                      for f in os.listdir(sites_folder) 
                      if f.endswith(".geojson")])

# Read and concatenate
sites_dfs = []
for f in sites_files:
    gdf = gpd.read_file(f)
    sites_dfs.append(gdf)

# Combine all sites_gdf
gdf_all_sites = gpd.GeoDataFrame(pd.concat(sites_dfs, ignore_index=False))  # KEEP original index

# Save the merged sites_gdf
gdf_all_sites.to_file("data/final/tree_sites_all.geojson", driver="GeoJSON")

print(f"Combined {len(sites_files)}")